# Get Atlas Release Dataset

## Imports

In [ ]:
import json
import rdflib
import getpass
import pandas as pd
from rdflib import RDF, RDFS, XSD, OWL, URIRef, BNode, SKOS
import pprint
from kgforge.core import KnowledgeGraphForge

## Setup
Get an authentication token

For now, the [Nexus web application](https://bbp.epfl.ch/nexus/web) can be used to get a token. We are looking for other simpler alternatives.

- Step 1: From the opened web page, click on the login button on the right corner and follow the instructions.

![login-ui](./login-ui.png)

- Step 2: At the end you’ll see a token button on the right corner. Click on it to copy the token.

![login-ui](./copy-token.png)


In [ ]:
TOKEN = getpass.getpass()

In [ ]:
endpoint_staging = "https://staging.nise.bbp.epfl.ch/nexus/v1"
endpoint_prod = "https://bbp.epfl.ch/nexus/v1"
endpoint=endpoint_prod

forge = KnowledgeGraphForge("https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/prod-forge-nexus.yml",
                            token=TOKEN,
                            endpoint=endpoint, 
                            bucket="bbp/atlas")

## Set the atlas release id
These atlas releases can be explored through the atlas web app:

* dev: https://bluebrainatlas.kcpdev.bbp.epfl.ch/atlas
* prod: https://bbp.epfl.ch/atlas


In [ ]:

Prod_BBP_Mouse_Brain_Atlas_Release = "https://bbp.epfl.ch/neurosciencegraph/data/4906ab85-694f-469d-962f-c0174e901885" 
Staging_BBP_Mouse_Brain_Atlas_Release = "https://bbp.epfl.ch/neurosciencegraph/data/brainatlasrelease/c96c71a8-4c0d-4bc1-8a1a-141d9ed6693d" 
atlas_release_id = Prod_BBP_Mouse_Brain_Atlas_Release

## Get the atlas release high level metadata

In [ ]:
atlas_release = forge.retrieve(atlas_release_id)

In [ ]:
print(atlas_release)

In [ ]:
atlas_release._store_metadata["_rev"]

## Get the atlas hierarchy

In [ ]:
parcellation_ontology = forge.retrieve(atlas_release.parcellationOntology.id, cross_bucket=True)

In [ ]:
print(parcellation_ontology)

In [ ]:
forge.download(parcellation_ontology, "distribution.contentUrl", ".", overwrite=True, cross_bucket=True)

## Get parcellation (annotation) volume

In [ ]:
parcellation_volume = forge.retrieve(atlas_release.parcellationVolume.id)

In [ ]:
print(parcellation_volume)

In [ ]:
forge.download(parcellation_volume, "distribution.contentUrl", ".", overwrite=True)

## Get orientation field volume

In [ ]:
query = {
          "type":"CellOrientationField", 
          "atlasRelease":{"@id":atlas_release_id},
          "brainLocation":{"brainRegion":{"id":"http://api.brain-map.org/api/v2/data/Structure/997"}} # root brain region
        }
cell_orientation_field = forge.search(query)
print(f"{len(cell_orientation_field)} found")

In [ ]:
print(cell_orientation_field)

In [ ]:
forge.download(cell_orientation_field, "distribution.contentUrl", ".", overwrite=True)

## Get the metype density nrrd file for each region (region is an input)

In [ ]:
resources = forge.search({"type":"METypeDensity","atlasRelease":{"@id":atlas_release_id}}, limit=1000, debug=False)
len(resources)

In [ ]:
reshaped_resources = forge.reshape(resources, keep=["id","type", "annotation.hasBody.id", "annotation.hasBody.label",
                     "brainLocation.brainRegion.id", "brainLocation.brainRegion.label", "distribution.atLocation.location"])
df = forge.as_dataframe(reshaped_resources, nesting="_")

df["mtype"] = df.apply(lambda row: (row.annotation[0]["hasBody"]["label"],row.annotation[0]["hasBody"]["id"]) , axis=1) 
df["etype"] = df.apply(lambda row: (row.annotation[1]["hasBody"]["label"],row.annotation[1]["hasBody"]["id"]) , axis=1) 
type_column = df.pop('type')
mtype_column = df.pop('mtype')
etype_column = df.pop('etype')

df.insert(0, 'type', type_column)
df.insert(1, 'mtype', mtype_column)
df.insert(2, 'etype', etype_column)
df.drop(columns="annotation")

print("")


In [ ]:
df.head(100)

In [ ]:
forge.download(resources, "distribution.contentUrl", ".", overwrite=True)